In [1]:
import tensorflow as tf 
import tensorflow_datasets as tfds
import numpy as np 
import jax 
import jax.numpy as jnp

# change the backend of keras to jax
import os
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import keras as k

2024-01-05 07:42:49.275601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 07:42:49.275649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 07:42:49.276738: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 07:42:49.283001: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/tqdm/

In [2]:
ds = tfds.load('mnist', split="train", shuffle_files=True)
ds = ds.shuffle(1024).batch(32).prefetch(tf.data.AUTOTUNE)
dsIter = iter(ds)

2024-01-05 07:42:52.006050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10514 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:17:00.0, compute capability: 6.1
2024-01-05 07:42:52.006629: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10523 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:65:00.0, compute capability: 6.1


In [3]:

def cnn():
    x = k.layers.Input([28, 28, 1])
    c1 = k.layers.Conv2D(32, (3,3), padding="same", activation=k.activations.elu)(x)
    c2 = k.layers.Conv2D(32, (3,3), padding="same", activation=k.activations.elu)(c1)
    
    flatten = k.layers.Flatten()(c2)
    
    fc1 = k.layers.Dense(512, k.activations.mish)(flatten)
    fc2 = k.layers.Dense(256, k.activations.mish)(fc1)
    out = k.layers.Dense(10, k.activations.mish)(fc2)
    
    return k.Model(inputs=x, outputs=out)

model = cnn()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)        │        320 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 32)        │      9,248 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 25088)             │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 512)               │ 12,845,568 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 256)               │    131,328 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_2 (Dense)                 │ (None, 10)                │      2,570 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 12,989,034 (49.55 MB)

 Trainable params: 12,989,034 (49.55 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
dsFetcher = next(dsIter)
img = (tf.cast(dsFetcher['image'], tf.float32) - 128)/128
model(img)

2024-01-05 07:42:52.409780: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2024-01-05 07:42:52.479804: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-05 07:42:52.602340: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


<tf.Tensor: shape=(32, 10), dtype=float32, numpy=
array([[ 0.02071405, -0.15030432, -0.13878724, -0.1331512 , -0.16221386,
         0.01276802, -0.0264954 , -0.03873387, -0.01070647,  0.17824659],
       [-0.12182382, -0.09863264, -0.0840658 , -0.0479426 , -0.09834213,
         0.07368324, -0.02340131,  0.02585653, -0.11278681, -0.01680233],
       [-0.00110923, -0.15035371, -0.15511146, -0.09201983, -0.11567782,
        -0.0318928 , -0.05272814, -0.03362067, -0.05143017,  0.05798335],
       [-0.06315028, -0.07172572, -0.14681861, -0.1103286 , -0.12290971,
         0.08926432, -0.05757037,  0.02709393, -0.04096495,  0.05826386],
       [-0.03422178, -0.12143856, -0.15902126, -0.06810808, -0.09093241,
         0.11487636,  0.02941085,  0.00426452, -0.03224974,  0.05307007],
       [ 0.0471933 , -0.05050071, -0.14925317, -0.11284717, -0.20329471,
         0.09013473, -0.11240883,  0.00215976, -0.09324394, -0.00485166],
       [-0.06528285, -0.11272226, -0.10264064, -0.05119326, -0.10393